In [14]:
import csv
import os
import torchvision
import torch.nn as nn
import torch
import torch.nn.functional as F
from torchvision import transforms, models, datasets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from torch import optim

import cv2, glob, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from torchvision import transforms, models, datasets
from torch_snippets import *
from torch_snippets import Report
from torchvision.ops import nms
from torch.utils.data import DataLoader, Dataset, Subset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import tkinter as tk
from tkinter import filedialog
import csv
import random

In [15]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(device)

cpu

In [16]:
# root = tk.Tk()
# root.withdraw()


# TEST_DATA_DIR = filedialog.askdirectory(
#     title="Select folder containing images for testing"
# )
TEST_DATA_DIR = "../test/"
print(TEST_DATA_DIR)
TEST_JSON_PATH = "../test/metadata.jsonl"
TEST_PREPROCESSED_CSV_PATH = "../test/annotations_test.csv"
MODEL_PATH = "../model/"

../test/

In [17]:
DF_TEST_IMAGE_DETECTIONS = pd.read_json(TEST_JSON_PATH, lines=True)
print(DF_TEST_IMAGE_DETECTIONS.head())

file_name                                            objects
0     0.png  {'bbox': [[100, 100, 110, 130], [80, 80, 90, 1...
1     1.png  {'bbox': [[100, 100, 110, 130], [80, 80, 90, 1...
2    10.png  {'bbox': [[100, 100, 110, 130], [80, 80, 90, 1...
3   100.png  {'bbox': [[100, 100, 110, 130], [80, 80, 90, 1...
4   101.png  {'bbox': [[100, 100, 110, 130], [80, 80, 90, 1...

In [18]:
def create_preprocessed_data(df=DF_TEST_IMAGE_DETECTIONS, csv_path=TEST_PREPROCESSED_CSV_PATH):
    
    with open(csv_path, 'w', newline='') as csvfile:
        writer_object = csv.writer(csvfile)
        writer_object.writerow(['ImageID','XMin','YMin','XMax','YMax','LabelName'])
    with open(csv_path, 'a', newline='') as csvfile:
        writer_object = csv.writer(csvfile)
        for i in range(len(df)):
            bboxes = df["objects"][i]["bbox"]
            categories = df["objects"][i]["categories"]
            assert(len(bboxes) == len(categories))
            for (bbox, category) in zip(bboxes, categories):
                new_row = []
                image_id = str(df["file_name"][i]).split(".")[0]
                new_row.append(image_id)
                x_min = bbox[0]
                y_min = bbox[1]
                x_max = bbox[2]
                y_max = bbox[3]
                new_row.extend([x_min, y_min, x_max, y_max, str(category)])
                writer_object.writerow(new_row)
                # print(new_row)


create_preprocessed_data(DF_TEST_IMAGE_DETECTIONS, TEST_PREPROCESSED_CSV_PATH)

In [19]:
DF_PREPROCESSED_TEST_IMAGE_DETECTIONS = pd.read_csv(TEST_PREPROCESSED_CSV_PATH)
print(DF_PREPROCESSED_TEST_IMAGE_DETECTIONS.head())

ImageID  XMin  YMin  XMax  YMax  LabelName
0        0   100   100   110   130          0
1        0    80    80    90   100          0
2        1   100   100   110   130          0
3        1    80    80    90   100          0
4       10   100   100   110   130          0

In [20]:
label2target = {l:t+1 for t,l in enumerate(DF_PREPROCESSED_TEST_IMAGE_DETECTIONS['LabelName'].unique())}
label2target['background'] = 0
target2label = {t:l for l,t in label2target.items()}
background_class = label2target['background']
num_classes = len(label2target)
print(num_classes)

2

In [21]:
def preprocess_image(img):
    img = torch.tensor(img).permute(2,0,1)
    return img.to(device).float().cpu()

In [22]:
class OpenDataset(torch.utils.data.Dataset):
    w, h = 224, 224
    def __init__(self, df, image_dir=TEST_DATA_DIR):
        self.image_dir = image_dir
        self.files = glob(self.image_dir+'/*.png')
        self.df = df
        self.image_infos = df.ImageID.unique()
    def __getitem__(self, ix):
        # load images and masks
        image_id = self.image_infos[ix]
        # img_path = find(str(image_id), self.files)
        img_path = os.path.join(self.image_dir, str(image_id) + ".png")
        print(img_path)
        img = Image.open(img_path).convert("RGB")
        img_w, img_h = img.size
        img = np.array(img.resize((self.w, self.h), resample=Image.BILINEAR))/255.
        data = self.df[self.df['ImageID'] == image_id]
        labels = data['LabelName'].values.tolist()
        labels = np.array(labels)
        boxes = data[['XMin','YMin','XMax','YMax']].values
        # boxes[:,[0,2]] *=  np.float64(self.w)/np.float64(img_w)
        # boxes[:,[1,3]] *=  np.float64(self.h)/np.float64(img_h)
        boxes[:,[0,2]] = ((boxes[:,[0,2]] * np.float64(self.w)) / np.float64(img_w))
        boxes[:,[1,3]] = ((boxes[:,[1,3]] * np.float64(self.h)) / np.float64(img_h))
        # boxes[:,[0,2]] /=  np.float64(img_w)
        # boxes[:,[1,3]] /=  np.float64(img_h)
        keep = (boxes[:, 3] > boxes[:, 1]) & (boxes[:, 2] > boxes[:, 0]) 
        boxes = boxes[keep]
        labels = labels[keep]
        boxes = boxes.astype(np.uint32).tolist() # convert to absolute coordinates
        if len(boxes) == 0:
          target = {}
          target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
          target["labels"] = torch.zeros((0, 1), dtype=torch.int64)
        else:
          # torch FRCNN expects ground truths as a dictionary of tensors
          target = {}
          target["boxes"] = torch.Tensor(boxes).float()
          target["labels"] = torch.Tensor([label2target[i] for i in labels]).long()
        img = preprocess_image(img)
        return img, target
    def collate_fn(self, batch):
        return tuple(zip(*batch)) 

    def __len__(self):
        return len(self.image_infos)

In [23]:
final_test_df = DF_PREPROCESSED_TEST_IMAGE_DETECTIONS

final_test_ds = OpenDataset(final_test_df)

final_test_loader = DataLoader(final_test_ds, batch_size=4, collate_fn=final_test_ds.collate_fn, drop_last=False)

# Inference

In [24]:
def get_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [25]:
model = get_model()
model.load_state_dict(torch.load(MODEL_PATH + "model_faster_rcnn_epoch15_ship_detector.pt"))

/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
def decode_output(output):
    'convert tensors to numpy arrays'
    bbs = output['boxes'].cpu().detach().numpy().astype(np.uint16)
    # print(bbs)
    labels = np.array([target2label[i] for i in output['labels'].cpu().detach().numpy()])
    confs = output['scores'].cpu().detach().numpy()
    ixs = nms(torch.tensor(bbs.astype(np.float32)), torch.tensor(confs), 0.05)
    bbs, confs, labels = [tensor[ixs] for tensor in [bbs, confs, labels]]

    if len(ixs) == 1:
        bbs, confs, labels = [np.array([tensor]) for tensor in [bbs, confs, labels]]
    return bbs.tolist(), confs.tolist(), labels.tolist()

In [ ]:
model.eval()
for ix, (images, targets) in enumerate(final_test_loader):
    if ix==3: break
    images = [im for im in images]
    outputs = model(images)
    print(outputs)

    for ix, output in enumerate(outputs):
        bbs, confs, labels = decode_output(output)
        info = [f'{l}@{c:.2f}' for l,c in zip(labels, confs)]
        show(images[ix].cpu().permute(1,2,0), bbs=bbs, texts=labels, sz=5)